# Go from matrices back to newick


### Outline:
[0. First Some Helper Functions](#section0)  
[1. Just Topology](#section1)  
[2. "Fuzzy" Toplogy](#section2)  
[3. Adding Branch Lengths](#section3)  
[4. Putting it all together](#section4)

### TODO:
- Add Branch Lenghts
- Combine into one function
- Test it out a bit to confirm it behaves basically as expected
- Add the slow messy version to the python code
- Add a few basic tests 
- Make theses functions faster (get rid of the nested loops, use numpy, etc)
- Clean the functions up and document better

In [1]:
from ete3 import Tree
import ast
import numpy as np
import pandas as pd

<a id="section0"></a>
## 0. Helper Functions

In [2]:
# Original Example I was using
exampleTopologyMatrix = [[0, 2, 3, 1, 3], [2, 0, 2, 2, 2], [3, 2, 0, 3, 1], [1, 2, 3, 0, 3], [3, 2, 1, 3, 0]]

# The '4Leaf.tree example' (used in step 3: adding branch lengths)
#exampleTopologyMatrix = [[0, 1, 2, 2], [1, 0, 2, 2], [2, 2, 0, 1], [2, 2, 1, 0]]

In [3]:
def matrixToDict(matrix, leafNames=False, rounding=False):
    '''Convert a symetrical matrix to a dictionary of dictonaries.'''
    if leafNames:
        leaves = leafNames
    else:
        leaves = [str(i) for i in range(len(matrix))]
    if rounding:
        matrix.round()
    dic = dict((key, value) for (key, value) in zip(leaves, matrix))    
    for key in dic:
        dic[key] = dict((key, value) for (key, value) in zip(leaves, dic[key]))
    return dic
    
# Test
#matrixToDict(exampleTopologyMatrix)

## Going from "flattened" symetrical matrix (list) back to the symetrical matrix (list of lists)

In [4]:
def flattenSymetricalMatrix(matrix):
    ''' Take a 2d symetrical matrix (list of lists).
        Return a flattened version (single list)
        for example:
           input [[0,2,3],
                  [2,0,4],
                  [3,4,0]]
           output [2,3,4]
                  '''
    flattenedMatrix = []
    for i in range(len(matrix)):
        if i == 0:
            pass
        else:
            for elm in matrix[i][0:i]:
                flattenedMatrix.append(elm)
        
    return flattenedMatrix
        
# Test
#flattenedList = flattenSymetricalMatrix(testMatrix)
#print(flattenedList)  

In [5]:
def triangleNumber(n):
    for i in range(n):
        if i*(i+1)/2 == n:
            return True
    return False

In [6]:
def unFlattenSymetricalMatrix(flattenedMatrix):
    ''' Take a flattened version (single list)
        return a 2d symetrical matrix (list of lists).
        
        for example:
           input  [1,2,3]
           output [[0,1,2],
                   [1,0,3],
                   [2,3,0]]

                  '''  
    if not triangleNumber(len(flattenedMatrix)):
        print('This Matrix cant be unflattened... its the wrong length')
        return
    
    unFlattenedMatrix = [[0]]
    node1 = 1
    node2 = 0
    for i in range(len(flattenedMatrix)):
        element = flattenedMatrix[i]
        
        # Add the element at node1, node2
        if len(unFlattenedMatrix) == node1:
            unFlattenedMatrix.append([element])
        else:
            unFlattenedMatrix[node1].append(element)
        # Add the element at ndoe2, node1 
        unFlattenedMatrix[node2].append(element)
        
        # Update the nodeIndices
        if (node1 - 1) == node2:
            unFlattenedMatrix[node1].append(0)
            node1+=1
            node2=0
        else:
            node2+=1

    return unFlattenedMatrix
            
# Test
# unFlattenSymetricalMatrix([1,2,3,4,5,6])
#This should return: [[0,1,2,4],[1,0,3,5],[2,3,0,6],[4,5,6,0]]

<a id="section1"></a>
## 1. Just topology

### From:
[[0, 2, 3, 1, 3],
[2, 0, 2, 2, 2],
[3, 2, 0, 3, 1],
[1, 2, 3, 0, 3],
[3, 2, 1, 3, 0]]

### To:
(3,5,(2,(1,4))) or some form thereof;

In [7]:
def createInternalNode(topologyDict, node1, node2):
    internalNodeName = "[" + node2 + "," + node1 + "]"
    topologyDict[internalNodeName] = {key: value-1 for (key, value) in topologyDict[node1].items()}
    del(topologyDict[node1])
    del(topologyDict[node2])
    for outerNode in topologyDict:
        del(topologyDict[outerNode][node1])
        del(topologyDict[outerNode][node2])
        if outerNode == internalNodeName:
            topologyDict[outerNode][internalNodeName] = 0
        else:
            topologyDict[outerNode][internalNodeName] = topologyDict[internalNodeName][outerNode]

    return topologyDict

# Test
#createInternalNode(exampleTopologyDict, '0', '3')

In [8]:
def topologyMatrixToList(topologyMatrix):
    topologyDict = matrixToDict(topologyMatrix)

    # Basically three loops:
    #   1. loop until the list is complete / the whole tree is traversed
    #   2. loop through rows
    #   3. loop through the columns of each row
    
    # Loop 1
    unresolvedInternalNodes = True
    while unresolvedInternalNodes:
        
        
        newRowsToInsert = []
        
        # Loop 2
        # Go through each row; not using the actual dict because it will be modified mid loop.
        rowKeys = list(topologyDict.keys())
        for rowKey in rowKeys:
            if rowKey in list(topologyDict.keys()): # To handle looping over a dict that's being modified.
                
                # Loop 3
                colKeys = list(topologyDict[rowKey].keys())
                for colKey in colKeys:
                    if rowKey in list(topologyDict.keys()) and colKey in list(topologyDict[rowKey].keys()): # To handle looping over a dict that's being modified.
                        
                        # Check if it is two nodes that share one internal node.
                        if topologyDict[rowKey][colKey] == 1:

                            # Add the internal node to the list
                            internalNodeName = "[" + rowKey + "," + colKey + "]"
                            internalNodeDict = {key: value-1 for (key, value) in topologyDict[rowKey].items()}
                            del(internalNodeDict[rowKey])
                            del(internalNodeDict[colKey])
                            newRowsToInsert.append({internalNodeName: internalNodeDict})
                            
                            # Remove the two rows representing the two nodes that were merged into the internal node.
                            del(topologyDict[rowKey])
                            del(topologyDict[colKey])
                            # Remove the two columns representing the two nodes that were merged (do this for each remaining row)
                            for remainingRowKey in topologyDict:
                                del(topologyDict[remainingRowKey][rowKey])
                                del(topologyDict[remainingRowKey][colKey])

        # After each time looping throught all rows.
        # Add the new internal node(s) to the topologyMatrix (both as a new row and a new column to each row)
        for newRow in newRowsToInsert:
            [(newRowKey, newRowVal)] = newRow.items()
            topologyDict[newRowKey] = newRowVal

            for row in topologyDict:
                if row in list(newRowVal.keys()):
                    topologyDict[row][newRowKey] = newRowVal[row]
                else:
                    topologyDict[row][newRowKey] = 0

        # if there are three or less nodes left than just combine these nodes and you have the topology list.
        numberOfUnresolvedNodes = len(list(topologyDict.keys()))
        if numberOfUnresolvedNodes <= 3:
            print('topologyDict: ', topologyDict)
            topologyListString = '['
            for nodeName in list(topologyDict.keys()):
                topologyListString += nodeName + ','
            topologyListString = topologyListString[:-1] + ']'
            unresolvedInternalNodes = False
            
    # Convert the topologyListString '[[1,2],[3,4]]' into an actual list [[1,2],[3,4]]
    topologyList = ast.literal_eval(topologyListString)
    
    return topologyList

    
# Test
topologyMatrixToList(exampleTopologyMatrix)

topologyDict:  {'[0,3]': {'[0,3]': 0, '[4,2]': 0, '4': 2, '1': 1, '2': 2}, '[4,2]': {'[4,2]': 0, '1': 1}, '1': {'[0,3]': 1, '[4,2]': 1, '1': 0}}


[[0, 3], [4, 2], 1]

<a id='section2'></a>
## 2. "Fuzzy" Toplogy
Non-integers and not perfectly describing one topology (i.e. conflicting info)

In [9]:
exampleFuzzy = [2.5746593, 2.9625423, 1.2669723, 1.8811567, 2.8555021, 2.5892396]
exampleTrue = [3, 3, 1, 1, 3, 3]
exampleFuzzy5 = [2.5746593, 2.9625423, 1.2669723, 1.8811567, 2.8555021, 2.5892396, 3.2, 2.4, 1.9, 4.1]

In [10]:
def replaceZerosWithInf(dictionary):
    modifiedDict = {}
    for rowKey in dictionary:
        modifiedRow = {}
        for colKey in dictionary[rowKey]:
            if dictionary[rowKey][colKey] == 0:
                modifiedRow[colKey] = float('inf')
            else:
                modifiedRow[colKey] = dictionary[rowKey][colKey]
        modifiedDict[rowKey] = modifiedRow
    return modifiedDict

In [11]:
def fuzzyTopologyMatrixToList(fuzzyTopologyMatrix):
    topologyDict = matrixToDict(fuzzyTopologyMatrix)
        
    # Replace the zero values along the diagonals with infinity
    # NOTE: THIS WILL REPLACE !!ALL!! ZEROS WITH INF
    topologyDict = replaceZerosWithInf(topologyDict)

    # Basically three loops:
    #   1. loop until the list is complete / the whole tree is traversed
    #   2. loop through rows
    #   3. loop through the columns of each row
    
    # Loop 1
    unresolvedInternalNodes = True
    while unresolvedInternalNodes:
        
        newRowsToInsert = []
        
        # Find the lowest number*
        minValue = float("inf")
        for rowKey in topologyDict:
            row = topologyDict[rowKey]
            keyMin = min(row.keys(), key=(lambda k: row[k]))
            rowMinValue = row[keyMin]
            if rowMinValue < minValue:
                minValue = rowMinValue
                minValueKeys = [rowKey, keyMin]

        # Add the internal node to the list
        rowKey = minValueKeys[0]
        colKey = minValueKeys[1]
        internalNodeName = "[" + rowKey + "," + colKey + "]"
        # Use the average of the dist between the two that were joined and the node of interest
        internalNodeDict1 = {key: value-1 for (key, value) in topologyDict[rowKey].items()}
        internalNodeDict2 = {key: value-1 for (key, value) in topologyDict[colKey].items()}
        df = pd.DataFrame([internalNodeDict1, internalNodeDict2])
        internalNodeDict = dict(df.mean())
        # Remove the two joined nodes from their list
        del(internalNodeDict[rowKey])
        del(internalNodeDict[colKey])
        newRowsToInsert.append({internalNodeName: internalNodeDict})

        # Remove the two rows representing the two nodes that were merged into the internal node.
        del(topologyDict[rowKey])
        del(topologyDict[colKey])
        # Remove the two columns representing the two nodes that were merged (do this for each remaining row)
        for remainingRowKey in topologyDict:
            del(topologyDict[remainingRowKey][rowKey])
            del(topologyDict[remainingRowKey][colKey])

        # After each time looping throught all rows.
        # Add the new internal node(s) to the topologyMatrix (both as a new row and a new column to each row)
        # TODO: Make it so that if there's a "tie" in min value they are both added
        for newRow in newRowsToInsert:
            [(newRowKey, newRowVal)] = newRow.items()
            topologyDict[newRowKey] = newRowVal

            for row in topologyDict:
                if row in list(newRowVal.keys()):
                    topologyDict[row][newRowKey] = newRowVal[row]
                else:
                    topologyDict[row][newRowKey] = float('inf')

        # if there are three or less nodes left than just combine these nodes and you have the topology list.
        numberOfUnresolvedNodes = len(list(topologyDict.keys()))
        if numberOfUnresolvedNodes <= 3:
            topologyListString = '['
            for nodeName in list(topologyDict.keys()):
                topologyListString += nodeName + ','
            topologyListString = topologyListString[:-1] + ']'
            unresolvedInternalNodes = False
            
    # Convert the topologyListString '[[1,2],[3,4]]' into an actual list [[1,2],[3,4]]
    topologyList = ast.literal_eval(topologyListString)
    
    return topologyList

    
# Test
fuzzyTopologyMatrixToList(unFlattenSymetricalMatrix(exampleFuzzy5))

[0, 3, [[1, 2], 4]]

<a id="section3"></a>
## 3. Adding Branch Lengths

Take a predicted newick string and a branch lenght matrix and go to a newick string with branch lenghts

### From:
[[2, 3], [1, 0]]  
and  
[[0, 0.361399, 0.39885499999999996, 0.497874],  
[0.361399, 0, 0.464316, 0.563335],  
[0.39885499999999996, 0.464316, 0, 0.430593],  
[0.497874, 0.5633349999999999, 0.430593, 0]]

### To:
((A:0.147969,B:0.213430):0.085099,C:0.165787,D:0.264806); or some form thereof;

In [12]:
testDistanceMatrix = [[0, 0.361399, 0.39885499999999996, 0.497874],
                      [0.361399, 0, 0.464316, 0.563335],
                      [0.39885499999999996, 0.464316, 0, 0.430593],
                      [0.497874, 0.5633349999999999, 0.430593, 0]]

testTopologyList = [[2, 3], [1, 0]]

In [13]:
def addBranchLengths(distanceMatrix, topologyList):
    return 'test'

In [14]:
addBranchLengths(testDistanceMatrix,testTopologyList)

'test'

1. Write out equations
2. Put it in a function